In [1]:
! pip install faiss-cpu==1.7.4 mistralai==0.0.12

In [2]:
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import googleapiclient.discovery

In [3]:
import methods 
import formats
import json
import os 
import re

from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage

google_calendar = methods.get_google_calendar_service()
calendar = methods.extractCalendar(google_calendar)
client = methods.set_up_MISTRAL(calendar)
now = methods.current_time()

Based on the provided calendar, there are no events scheduled for today (2024-04-01) that match the format you have given.

In [7]:
context = f"Here's my calendar for the week, store it for your reference. Today's date and time is: {now}\n{calendar}"
instruction = """\nAs my virtual assistant, you are an expert with managing the calendar.
You can create, move, check, list, update, delete, and handle recurring events based on the user's requests.
When the user doesn't provide enough information, you can ask for more details, upto 3 times. Otherwise use your best judgement. 
When the user wants to create an event or move an event and if the user hasn't specified when they want it scheduled, then refer to the calendar and use your best judgement to schedule the event. Keep in mind when the user would normally eat or sleep. Assume default duration for any event as 1 hour.
Check for any conflicts before scheduling an event.
Always ask for confirmation before making any changes to the calendar, if the user doesn't agree, if the user doesn't agree, then generate another suitable time slot.
When you are done, please let the user know that the task has been completed, and don't ask any more questions. If you aren't done, end with a question mark.
Please perform the necessary actions using the Google Calendar API.\n"""

# initial setup
initial_prompt = context + instruction

instruction2 = """Today's date and time is: {now}\n{calendar}\\nBased on the user's request you've just processed, please output the appropriate function call(s) to interact with the Google Calendar API. Format your response as a single string containing one or more function calls separated by semicolons. Do not include explanations or additional text. 
All dates should be in 'YYYY-MM-DD' format, and times should be in 24-hour format 'HH:MM'. Additionally, for functions requiring a datetime range (such as 'checkSchedule' and 'ListEvents'), the datetime should be in ISO 8601 format 'YYYY-MM-DDTHH:MM:SSZ'.
Available function templates to include in the response string:
- createEvent(description, date:default=today, time, duration:default=1)
- moveEvent(event_id, new_date, new_time)
- updateEvent(event_id, new_description, new_duration, new_location)
- deleteEvent(event_id)
- createRecurringEvent(description, date, time, duration, frequency, until)
- deleteRecurringEvent(event_id)
- updateRecurringEvent(event_id, new_description, new_duration, new_location)

Make sure to fill in the required information, such as event names, IDs, dates, times, etc., based on the user's request context. Replace placeholders like 'today' or 'tomorrow' with specific dates in the format YYYY-MM-DD. If more details are needed from the user for a function call, use your best judgment to complete the function with the information available.

Example input: 'Schedule a team meeting for Friday afternoon at 3pm'
Example output: 'createEvent("Team Meeting", "2024-04-05", "15:00", 1)'

Now, please generate the necessary function call(s) based on the user's request."""
user_request = "Can you schedule a meeting for me tomorrow at 9 PM?"

messages = [
    ChatMessage(
        role="system",
        content=initial_prompt  # System message with instructions and context
    ),
    ChatMessage(
        role="system",
        content=instruction2  # System message with instructions and context
    ),
    ChatMessage(
        role="user",
        content=user_request  # User's initial request
    )
]

stream_response = client.chat_stream(model="open-mistral-7b", messages=messages)

function_call = ''
for chunk in stream_response:
    function_call += chunk.choices[0].delta.content or ""
print(function_call)


createEvent("Meeting", "2024-04-02", "21:00", 1)


In [8]:
function_calls = [line.strip() for line in function_call.split(';')] 
print(function_calls)

# Define a dictionary mapping function names to their corresponding Python functions
functions = {
    "createEvent": methods.createEvent,
    "moveEvent": methods.moveEvent,
    "checkSchedule": methods.checkSchedule,
}

#TODO: works for createEvent, gotta change other functions in methods file to accept arguments as strings and convert them to appropriate data types
# Iterate through the function calls
for call in function_calls:
    # Split the string to extract function name and arguments
    parts = call.split('(')
    function_name = parts[0]
    arguments = parts[1].rstrip(')').split(', ')
    
    # Call the corresponding function with the provided arguments
    if function_name in functions:
        functions[function_name](*arguments)
    else:
        print(f"Function '{function_name}' not found.")

['createEvent("Meeting", "2024-04-02", "21:00", 1)']
Create event called
The data and time is  2024-04-02 / 21:00
UTC start time:  2024-04-03 01:00:00+00:00 UTC end time:  2024-04-03 02:00:00+00:00
Event created: Meeting on 2024-04-02 at 21:00 for duration of 1 hours.
